In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as ms
import xgboost as xgb
import bisect

from scipy import stats
from tqdm import tqdm
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import RandomForestRegressor

In [ ]:
train = pd.read_csv(r'C:\Users\dlwks\OneDrive\바탕 화면\VSCode\HD_CNC\train.csv').drop(['X1'], axis=1)
test = pd.read_csv(r'C:\Users\dlwks\OneDrive\바탕 화면\VSCode\HD_CNC\test.csv').drop(['Id', 'X1'], axis=1)

display(train.head())
display(test.head())

| 항목 | 설명     |
|-----|----------|
| X1  | 작업번호 |
| X2  | 마킹길이 |
| X3  | 절단길이 |
| X4  | 철판두께 |
| X5  | 철판재질 |
| X6  | 절단갯수 |
| X7  | 작업장   |
| X8  | 작업자   |
| Y1  | 마킹시간 |
| Y2  | 절단시간 |

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
train['X8'] = train['X8'].str.extract(r'(PL|PS)')
test['X8'] = test['X8'].str.extract(r'(PL|PS)')

In [ ]:
train['X8_encoded'] = (train['X8'] != 'PL').astype(int)
test['X8_encoded'] = (test['X8'] != 'PL').astype(int)

train = pd.get_dummies(train, columns=['X8_encoded'], prefix=['X8'])
test = pd.get_dummies(test, columns=['X8_encoded'], prefix=['X8'])

train[['X8_0', 'X8_1']] = train[['X8_0', 'X8_1']].astype(int)
test[['X8_0', 'X8_1']] = test[['X8_0', 'X8_1']].astype(int)

In [ ]:
train['X5'].unique()

In [ ]:
import re

def convert_x5(value):
    # -TM 이 포함되어 있으면 제거하고 숫자로 변환, 그렇지 않으면 그냥 숫자로 변환
    if '-TM' in value:
        value = value.replace('-TM', '')
    match = re.search(r'\d+', value)  # 정규표현식을 사용하여 숫자 추출
    if match:
        return int(match.group()) * 9.8
    else:
        return 235  # 숫자가 없는 경우 235로 설정

# X5 열에 변환 함수 적용
train['X5_numeric'] = train['X5'].apply(convert_x5)
test['X5_numeric'] = test['X5'].apply(convert_x5)

In [ ]:
train['X5'].unique()

In [ ]:
train['X5_numeric'].unique()

In [ ]:
train_y1 = np.log1p(train['Y1'])
train_y2 = np.log1p(train['Y2'])

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

selected_columns = ['X2', 'X3', 'X4', 'X6', 'X7']

scaler = StandardScaler()

train[selected_columns] = scaler.fit_transform(train[selected_columns])
test[selected_columns] = scaler.transform(test[selected_columns])

In [ ]:
# DBSCAN 모델 생성 및 적용
dbscan = DBSCAN(eps=0.5, min_samples=5)
train['cluster'] = dbscan.fit_predict(train[selected_columns])

# 클러스터가 -1인 것은 이상치로 간주하고 제거
filtered_train = train[train['cluster'] != -1].drop(columns=['cluster'])

filtered_train

In [ ]:
filtered_train.drop(columns = ['X5', 'X8'], inplace = True)

In [ ]:
xgb_y1 = xgb.XGBRegressor(eval_metric='mae',
                          enable_categorical=True 
                          )

X_train_reduced = filtered_train.reset_index(drop=True)
y_train = train_y1.reset_index(drop=True)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

ensemble_predictions = []
scores = []

y_train = y_train.values

for train_idx, val_idx in tqdm(kf.split(X_train_reduced), total=5, desc="Processing folds"):
    X_t, X_val = X_train_reduced.iloc[train_idx], X_train_reduced.iloc[val_idx]
    y_t, y_val = y_train[train_idx], y_train[val_idx]

    xgb_y1.fit(X_t, y_t, eval_set=[(X_val, y_val)], verbose=50)

    val_pred = xgb_y1.predict(X_val)

    scores.append(mean_absolute_error(y_val, val_pred))

    test = test[X_train_reduced.columns]

    xgb_pred = xgb_y1.predict(test)
    xgb_pred = np.where(xgb_pred < 0, 0, xgb_pred)

    ensemble_predictions.append(xgb_pred)

final_predictions_y1 = np.mean(ensemble_predictions, axis=0)

print("Validation : MAE scores for each fold:", scores)
print("Validation : MAE:", np.mean(scores))

In [ ]:
xgb_y2 = xgb.XGBRegressor(eval_metric='mae',
                          enable_categorical=True 
                          )

X_train_reduced = filtered_train.reset_index(drop=True)
y_train = train_y2.reset_index(drop=True)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

ensemble_predictions = []
scores = []

y_train = y_train.values

for train_idx, val_idx in tqdm(kf.split(X_train_reduced), total=5, desc="Processing folds"):
    X_t, X_val = X_train_reduced.iloc[train_idx], X_train_reduced.iloc[val_idx]
    y_t, y_val = y_train[train_idx], y_train[val_idx]

    xgb_y2.fit(X_t, y_t, eval_set=[(X_val, y_val)], verbose=50)

    val_pred = xgb_y2.predict(X_val)

    scores.append(mean_absolute_error(y_val, val_pred))

    xgb_pred = xgb_y2.predict(test)
    xgb_pred = np.where(xgb_pred < 0, 0, xgb_pred)

    ensemble_predictions.append(xgb_pred)

final_predictions_y2 = np.mean(ensemble_predictions, axis=0)

print("Validation : MAE scores for each fold:", scores)
print("Validation : MAE:", np.mean(scores))

In [ ]:
submit = pd.read_csv(r'C:\Users\dlwks\OneDrive\바탕 화면\VSCode\HD_CNC\sample_submission.csv')

submit.head()

In [ ]:
final_predictions = (np.exp(final_predictions_y1) - 1) + (np.exp(final_predictions_y2) - 1)


submit.iloc[:, 1] = final_predictions

submit

In [ ]:
submit.to_csv('C:\\Users\\dlwks\\OneDrive\\바탕 화면\\VSCode\\HD_CNC\\1115-1.csv', index=False)

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import pandas as pd

# # Assuming your DataFrame is named 'train'
# # If not, replace 'train' with the actual name of your DataFrame

# # Set the style for Seaborn
# sns.set(style="whitegrid")

# # Set up the matplotlib figure
# fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(15, 10))

# # Flatten the axes for easier iteration
# axes = axes.flatten()

# # Loop through each column and plot the distribution
# for i, column in enumerate(train.columns):
#     sns.histplot(train[column], kde=True, ax=axes[i], bins=30)
#     axes[i].set_title(f'Distribution of {column}')

# # Adjust layout
# plt.tight_layout()
# plt.show()

In [ ]:
mmtime = train.groupby('X5')['Y1'].mean()
mctime = train.groupby('X5')['Y2'].mean()

wmtime = train.groupby('X7')['Y1'].mean()
wctime = train.groupby('X7')['Y2'].mean()

manmtime = train.groupby('X8')['Y1'].mean()
manctime = train.groupby('X8')['Y2'].mean()

train['X5_mmtime'] = train['X5'].map(mmtime)
train['X5_mctime'] = train['X5'].map(mctime)

train['X7_wmtime'] = train['X7'].map(wmtime)
train['X7_wctime'] = train['X7'].map(wctime)

train['X8_manmtime'] = train['X8'].map(manmtime)
train['X8_manctime'] = train['X8'].map(manctime)

test['X5_mmtime'] = test['X5'].map(mmtime)
test['X5_mctime'] = test['X5'].map(mctime)

test['X7_wmtime'] = test['X7'].map(wmtime)
test['X7_wctime'] = test['X7'].map(wctime)

test['X8_manmtime'] = test['X8'].map(manmtime)
test['X8_manctime'] = test['X8'].map(manctime)

In [ ]:
bins = [0, 15, 40, 70, 95, 110, float('inf')]

labels = [0, 1, 2, 3, 4, 5]

train['X6_category'] = pd.cut(train['X6'], bins = bins, labels = labels, include_lowest = True)
test['X6_category'] = pd.cut(test['X6'], bins = bins, labels = labels, include_lowest = True)

train.drop(columns = ['X6'], inplace = True)
test.drop(columns = ['X6'], inplace = True)

sns.scatterplot(x=train['X2'], y=train['X3'], hue = train['X6_category'].astype('category'))

In [ ]:
train